In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import matplotlib.lines as mlines
import numpy as np
from scipy.signal import savgol_filter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Normalizer
import re
import csv

# Paramétrage de l'affichage des dataframes
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', None)

In [ ]:
# Détermination du temps de vieillissement à partir du nom de fichier
def age(parameters):    
    compound_mapping = {
        'T0': 0,
        'T1': 14,
        'T2': 33,
        'T3': 42,
        'T4': 61,
        'T5': 80,
        '15122023': 294,
        '19012024': 329,
        '08022024': 349,
        '23022024': 364,
        '08032024': 378,
        '22032024': 392,
        '05042024': 406,
        '19042024': 420,
        '03052024': 434,
        '17052024': 448,
        '07062024': 462

    }
    
    for key, value in compound_mapping.items():
        if key in parameters:
            return value
    return 0



In [ ]:
# Détermination du type de matériau à partir du nom de fichier
# Tient compte de toutes les nomenclatures utilisées pendant 
# les stages SPECTRAA et DEGRAA-D
def compound(filename):
    parameters = re.split(r'[-_. ]', filename)
    compound_mapping = {
        ('12', 'a'): 'Caoutchouc',
        ('12', 'su'): 'PVC',
        ('13', 'a'): 'Acrylique',
        ('13', 'su'): 'AC',
        ('14', 'a'): 'Caoutchouc',
        ('14', 'su'): 'PP',
        ('15', 'a'): 'Acrylique',
        ('15', 'su'): 'PET',
        ('16', 'a'): 'Acrylique',
        ('16', 'su'): 'PP',
        ('12', 'fa'): 'Caoutchouc',
        ('12', 'su'): 'PVC',
        ('13', 'fa'): 'Acrylique',
        ('13', 'su'): 'AC',
        ('14', 'fa'): 'Caoutchouc',
        ('14', 'su'): 'PP',
        ('15', 'fa'): 'Acrylique',
        ('15', 'su'): 'PET',
        ('16', 'fa'): 'Acrylique',
        ('16', 'su'): 'PP',
        ('RAA12', 'MAPrelevement'): 'Caoutchouc',
        ('RAA12', 'FilmsPrelevement'): 'PVC',
        ('RAA13', 'MAPrelevement'): 'Acrylique',
        ('RAA13', 'FilmsPrelevement'): 'AC',
        ('RAA14', 'MAPrelevement'): 'Caoutchouc',
        ('RAA14', 'FilmsPrelevement'): 'PP',
        ('RAA15', 'MAPrelevement'): 'Acrylique',
        ('RAA15', 'FilmsPrelevement'): 'PET',
        ('RAA16', 'MAPrelevement'): 'Acrylique',
        ('RAA16', 'FilmsPrelevement'): 'PP',
        ('RAA12', 'fa'): 'Caoutchouc',
        ('RAA12', 'fna'): 'PVC',
        ('RAA13', 'fa'): 'Acrylique',
        ('RAA13', 'fna'): 'AC',
        ('RAA14', 'fa'): 'Caoutchouc',
        ('RAA14', 'fna'): 'PP',
        ('RAA15', 'fa'): 'Acrylique',
        ('RAA15', 'fna'): 'PET',
        ('RAA16', 'fa'): 'Acrylique',
        ('RAA16', 'fna'): 'PP',
        ('RAA12', 'fa'): 'Caoutchouc',
        ('RAA12', 'fna'): 'PVC',
        ('RAA13', 'fa'): 'Acrylique',
        ('RAA13', 'fna'): 'AC',
        ('RAA14', 'fa'): 'Caoutchouc',
        ('RAA14', 'fna'): 'PP',
        ('RAA15', 'fa'): 'Acrylique',
        ('RAA15', 'fna'): 'PET',
        ('RAA16', 'fa'): 'Acrylique',
        ('RAA16', 'fna'): 'PP',
        ('RAA12', 'MA'): 'Caoutchouc',
        ('RAA12', 'FILM'): 'PVC',
        ('RAA12', 'Film'): 'PVC',
        ('RAA13', 'MA'): 'Acrylique',
        ('RAA13', 'FILM'): 'AC',
        ('RAA13', 'Film'): 'AC',
        ('RAA14', 'MA'): 'Caoutchouc',
        ('RAA14', 'FILM'): 'PP',
        ('RAA14', 'Film'): 'PP',
        ('RAA15', 'MA'): 'Acrylique',
        ('RAA15', 'FILM'): 'PET',
        ('RAA15', 'Film'): 'PET',
        ('RAA16', 'MA'): 'Acrylique',
        ('RAA16', 'FILM'): 'PP',
        ('RAA16', 'Film'): 'PP'
    }
    
    for key, value in compound_mapping.items():
        if key[0] in parameters and key[1] in parameters:
            return value
    return filename

In [ ]:
# Indiquer l'emplacement des données
directory =

ages = []  # List to store age values
filenames = []   # List to store filenames
tapes = [] #List to store tape type
compounds = [] 
dataframes = []  # List to store individual DataFrame for each file

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path, skiprows=2, delimiter=';', names=['Wavelength', 'Absorbance'], dtype={'Wavelength': int, 'Absorbance': float}, index_col='Wavelength', decimal=',') #
        parameters = re.split(r'[-_. ]', filename)
        with open(file_path, 'r') as file: #Check for A or %T
            csv_reader = csv.reader(file, delimiter=';')
            next(csv_reader)  # Skip the first row
            row = next(csv_reader)  # Read the second row
            value = row[1]  # Extract the value from the second column
        if value == '%T' : # Conversion des données de transmitance en absorbance
            df['Absorbance'] = -np.log10(df['Absorbance']/100) #apply 2-log(%T) transformation
        tapes.append(parameters[0]) # Extrait le numéro de RAA du nom de fichier
        ages.append(age(parameters)) # Extrait le temps de vieillissement du nom de fichier
        filenames.append(filename) # Extrait le nom de fichier
        compounds.append(compound(filename)) # Extrait le type de matériau du nom de fichier
        dataframes.append(df)

# Concatenate all DataFrames and metadata
data = pd.concat(dataframes, axis=1)

# Create the final DataFrame
data = data.T.reset_index(drop=True).rename_axis('Index')

data['compound'] = compounds
data['age'] = ages
data['file name'] = filenames
data['tape'] = tapes

cols = list(data.columns)
cols = ['compound'] + [col for col in cols if col != 'compound']
data = data[cols]

data.reset_index(drop=True, inplace=True)

data = data.sort_values(by=['compound', 'age', 'file name'])
data.reset_index(drop=True, inplace=True)
data

In [ ]:
# Dérivation des données et/ou applique une normalisation ou standardisation
def extract_data(data, deriv, normalization='standard'):
    lab = data.values[:, -1]
    lab2 = data.values[:, 0]
    labs = [lab, lab2]
    feat = data.values[:, 2:-3].astype('float32')
    feat = feat[:, ::-1]
    
    if normalization == 'standard':
        scaler = StandardScaler()
    elif normalization == 'normalise':
        scaler = Normalizer()
    else:
        scaler = None
    
    if scaler:
        nfeat = scaler.fit_transform(feat)
    else:
        nfeat = feat
    
    if deriv == 1:
        dfeat = savgol_filter(feat, 25, polyorder=5, deriv=1)
        if scaler:
            ndfeat = scaler.fit_transform(dfeat)
        else:
            ndfeat = dfeat
        return ndfeat, labs
    elif deriv == 2:
        ddfeat = savgol_filter(feat, 25, polyorder=5, deriv=2)
        if scaler:
            nddfeat = scaler.fit_transform(ddfeat)
        else:
            nddfeat = ddfeat
        return nddfeat, labs
    
    return nfeat, labs

In [ ]:
# Application de l'ACP
# Il aurait été intéressant d'essayer d'autres algorithmes que le
# SVD, par exemple Nipals
def apply_PCA_4(nfeat, method) :

    if method == 'SVD':
        pca = PCA(n_components=4, svd_solver='full')
        Xt = pca.fit_transform(nfeat)
    else:
        raise ValueError("Invalid method. Please choose either 'SVD' or 'NIPALS'.")

    return Xt, pca

In [ ]:
# Afficher les performances de l'ACP
def test_PCA(data):
    nfeat1, labs1 = extract_data(data, 0)
    nfeat2, labs2 = extract_data(data, 1)
    
    # Initialise
    PCA1 = PCA(n_components = 10, svd_solver='full')
    PCA2 = PCA(n_components = 10, svd_solver='full')
    
    # Fit the spectral data and extract the explained variance ratio
    X1 = PCA1.fit(nfeat1)
    expl_var_1 = X1.explained_variance_ratio_

    # Fit the first derrivative data and extract the explained variance ratio
    X2 = PCA2.fit(nfeat2)
    expl_var_2 = X2.explained_variance_ratio_

    # Plot data
    with plt.style.context('ggplot'):
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(9, 6))
        fig.set_tight_layout(True)

        ax1.plot(expl_var_1, '-o', label = "Explained Variance %")
        ax1.plot(np.cumsum(expl_var_1),'-o', label='Cumulative variance %')
        ax1.set_xlabel("PC number")
        ax1.set_title("Absorbance data")

        ax2.plot(expl_var_2, '-o', label="Explained Variance %")
        ax2.plot(np.cumsum(expl_var_2), '-o', label='Cumulative variance %')
        ax2.set_xlabel("PC number")
        ax2.set_title("First derivative data")

        plt.legend()
        plt.show()

In [ ]:
# Même fonction que test_PCA mais sans afficher les courbes
# afin d'extraire ??
def test_PCA_nofig(data):
    nfeat1, labs1 = extract_data(data, 0)
    nfeat2, labs2 = extract_data(data, 1)
    
    # Initialise
    PCA1 = PCA(n_components = 10, svd_solver='full')
    PCA2 = PCA(n_components = 10, svd_solver='full')
    
    # Fit the spectral data and extract the explained variance ratio
    X1 = PCA1.fit(nfeat1)
    expl_var_1 = X1.explained_variance_ratio_
    print(expl_var_1)
    # Fit the first derrivative data and extract the explained variance ratio
    X2 = PCA2.fit(nfeat2)
    expl_var_2 = X2.explained_variance_ratio_
    

In [ ]:
test_PCA(data)

In [ ]:
# Représente l'ACP en fonction de PC1 et PC2
# Avec variation de couleur entre les temps de vieillissement
def plot_PCA_map_age_var(Xt, labs, data):    
    lab, lab2 = labs
    unique1 = sorted(set(lab))
    unique2 = sorted(set(lab2))
    ages = data.values[:, -3]

    with plt.style.context('ggplot'):
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        
        ax = axs[0]
        handles = []
        
        for u in unique1:
            for i in range(len(Xt[lab == u, 0])):
                xi = Xt[lab == u, 0]
                yi = Xt[lab == u, 1]

                filtered_ages = ages[lab == u]
                non_zero_ages = filtered_ages[filtered_ages > 0]

                if len(non_zero_ages) > 1:
                    second_min_age = np.sort(non_zero_ages)[0]
                    ages_variation = (filtered_ages - second_min_age) / (filtered_ages.max() - second_min_age)
                else:
                    ages_variation = np.zeros_like(filtered_ages)

                color_map_idx = np.round(ages_variation[i] * 255).astype(int)

                if u == 'RAA12':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.1, 0.4, 256))
                if u == 'RAA13':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.9, 0.6, 256))
                if u == 'RAA14':
                    adjusted_color = plt.cm.RdGy(np.linspace(0.9, 0.6, 256))
                if u == 'RAA15':
                    adjusted_color = plt.cm.BrBG(np.linspace(0.9, 0.6, 256))
                if u == 'RAA16':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.1, 0.4, 256))
                if u == 'Caoutchouc':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.1, 0.4, 256))
                if u == 'Acrylique':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.9, 0.6, 256))
                if u == 'PVC':
                    adjusted_color = plt.cm.RdGy(np.linspace(0.9, 0.6, 256))
                if u == 'AC':
                    adjusted_color = plt.cm.BrBG(np.linspace(0.9, 0.6, 256))
                if u == 'PP':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.1, 0.4, 256))
                if u == 'PET':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.9, 0.6, 256))

                # Plot only one label for each unique type
                edge_width = 0.2
                if filtered_ages[i] == 0:
                    color = adjusted_color[np.round((second_min_age - second_min_age) / (filtered_ages.max() - second_min_age) * 255).astype(int)]
                else:
                    color = adjusted_color[color_map_idx]

                if i == 0:
                    ax.scatter(xi[i], yi[i], color=color, s=75, edgecolors='k', linewidths=edge_width, label=str(u), alpha=0.7)
                    handles.append(mlines.Line2D([], [], color=adjusted_color[75], alpha=0.7, marker='o', linestyle='None', label=str(u)))
                else:
                    ax.scatter(xi[i], yi[i], color=color, s=75, edgecolors='k', linewidths=edge_width, alpha=0.7)

        ax.set_xlabel('PC1', fontsize='x-large')
        ax.set_ylabel('PC2', fontsize='x-large')
        ax.legend(loc='upper right', handles=handles, fontsize=12, markerscale=1.2)
        
        ax = axs[1]

        handles = []
        for u in unique2:
            for i in range(len(Xt[lab2 == u, 0])):
                xi = Xt[lab2 == u, 0]
                yi = Xt[lab2 == u, 1]

                filtered_ages = ages[lab2 == u]
                non_zero_ages = filtered_ages[filtered_ages > 0]

                if len(non_zero_ages) > 1:
                    second_min_age = np.sort(non_zero_ages)[0]
                    ages_variation = (filtered_ages - second_min_age) / (filtered_ages.max() - second_min_age)
                else:
                    ages_variation = np.zeros_like(filtered_ages)

                color_map_idx = np.round(ages_variation[i] * 255).astype(int)

                if u == 'RAA12':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.1, 0.4, 256))
                if u == 'RAA13':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.9, 0.6, 256))
                if u == 'RAA14':
                    adjusted_color = plt.cm.RdGy(np.linspace(0.9, 0.6, 256))
                if u == 'RAA15':
                    adjusted_color = plt.cm.BrBG(np.linspace(0.9, 0.6, 256))
                if u == 'RAA16':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.1, 0.4, 256))
                if u == 'Caoutchouc':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.1, 0.4, 256))
                if u == 'Acrylique':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.9, 0.6, 256))
                if u == 'PVC':
                    adjusted_color = plt.cm.RdGy(np.linspace(0.9, 0.6, 256))
                if u == 'AC':
                    adjusted_color = plt.cm.BrBG(np.linspace(0.9, 0.6, 256))
                if u == 'PP':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.1, 0.4, 256))
                if u == 'PET':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.9, 0.6, 256))

                edge_width = 0.2
                if filtered_ages[i] == 0:
                    color = adjusted_color[np.round((second_min_age - second_min_age) / (filtered_ages.max() - second_min_age) * 255).astype(int)]
                else:
                    color = adjusted_color[color_map_idx]
                if i == 0:
                    ax.scatter(xi[i], yi[i], color=color, s=75, edgecolors='k', linewidths=edge_width, label=str(u), alpha=0.7)
                    handles.append(mlines.Line2D([], [], color=adjusted_color[75], alpha=0.7, marker='o', linestyle='None', label=str(u)))
                else:
                    ax.scatter(xi[i], yi[i], color=color, s=75, edgecolors='k', linewidths=edge_width, alpha=0.7)

        ax.set_xlabel('PC1', fontsize='x-large')
        ax.set_ylabel('PC2', fontsize='x-large')
        ax.legend(loc='upper right', handles=handles, fontsize=12, markerscale=1.2)
        plt.tight_layout()
        plt.show()

In [ ]:
# Représente l'ACP en fonction de PC1 et PC2
# Pas de variation de couleur entre les temps de vieillissement
def plot_PCA_map_no_age_var(Xt, labs, data):    
    lab, lab2 = labs
    unique1 = sorted(set(lab))
    unique2 = sorted(set(lab2))

    ages = data.values[:, -3]

    with plt.style.context('ggplot'):
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        
        ax = axs[0]
        handles = []
        
        for u in unique1:
            for i in range(len(Xt[lab == u, 0])):
                xi = Xt[lab == u, 0]
                yi = Xt[lab == u, 1]

                filtered_ages = ages[lab == u]
                non_zero_ages = filtered_ages[filtered_ages > 0]

                if len(non_zero_ages) > 1:
                    second_min_age = np.sort(non_zero_ages)[0]
                    ages_variation = (filtered_ages - second_min_age) / (filtered_ages.max() - second_min_age)
                else:
                    ages_variation = np.zeros_like(filtered_ages)

                color_map_idx = np.round(ages_variation[i] * 255).astype(int)

                if u == 'RAA12':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.1, 0.4, 256))
                if u == 'RAA13':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.9, 0.6, 256))
                if u == 'RAA14':
                    adjusted_color = plt.cm.RdGy(np.linspace(0.9, 0.6, 256))
                if u == 'RAA15':
                    adjusted_color = plt.cm.BrBG(np.linspace(0.9, 0.6, 256))
                if u == 'RAA16':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.1, 0.4, 256))
                if u == 'Caoutchouc':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.1, 0.4, 256))
                if u == 'Acrylique':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.9, 0.6, 256))
                if u == 'PVC':
                    adjusted_color = plt.cm.RdGy(np.linspace(0.9, 0.6, 256))
                if u == 'AC':
                    adjusted_color = plt.cm.BrBG(np.linspace(0.9, 0.6, 256))
                if u == 'PP':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.1, 0.4, 256))
                if u == 'PET':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.9, 0.6, 256))

                edge_width = 0.2
                if filtered_ages[i] == 0:
                    color = adjusted_color[100]
                else:
                    color = adjusted_color[100]

                if i == 0:
                    ax.scatter(xi[i], yi[i], color=color, s=75, edgecolors='k', linewidths=edge_width, label=str(u), alpha=0.7)
                    handles.append(mlines.Line2D([], [], color=adjusted_color[75], alpha=0.7, marker='o', linestyle='None', label=str(u)))
                else:
                    ax.scatter(xi[i], yi[i], color=color, s=75, edgecolors='k', linewidths=edge_width, alpha=0.7)

        ax.set_xlabel('PC1 (71,2 %)', fontsize='x-large')
        ax.set_ylabel('PC2 (10,4 %)', fontsize='x-large')
        ax.legend(loc='upper right', handles=handles, fontsize=12, markerscale=1.2)
        
        ax = axs[1]

        handles = []
        for u in unique2:
            for i in range(len(Xt[lab2 == u, 0])):
                i_fixed = 0
                xi = Xt[lab2 == u, 0]
                yi = Xt[lab2 == u, 1]

                filtered_ages = ages[lab2 == u]
                non_zero_ages = filtered_ages[filtered_ages > 0]

                if len(non_zero_ages) > 1:
                    second_min_age = np.sort(non_zero_ages)[0]
                    ages_variation = (filtered_ages - second_min_age) / (filtered_ages.max() - second_min_age)
                else:
                    ages_variation = np.zeros_like(filtered_ages)

                color_map_idx = np.round(ages_variation[i] * 255).astype(int)

                if u == 'RAA12':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.1, 0.4, 256))
                if u == 'RAA13':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.9, 0.6, 256))
                if u == 'RAA14':
                    adjusted_color = plt.cm.RdGy(np.linspace(0.9, 0.6, 256))
                if u == 'RAA15':
                    adjusted_color = plt.cm.BrBG(np.linspace(0.9, 0.6, 256))
                if u == 'RAA16':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.1, 0.4, 256))
                if u == 'Caoutchouc':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.1, 0.4, 256))
                if u == 'Acrylique':
                    adjusted_color = plt.cm.PiYG(np.linspace(0.9, 0.6, 256))
                if u == 'PVC':
                    adjusted_color = plt.cm.RdGy(np.linspace(0.9, 0.6, 256))
                if u == 'AC':
                    adjusted_color = plt.cm.BrBG(np.linspace(0.9, 0.6, 256))
                if u == 'PP':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.1, 0.4, 256))
                if u == 'PET':
                    adjusted_color = plt.cm.PuOr(np.linspace(0.9, 0.6, 256))

                edge_width = 0.2
                if filtered_ages[i] == 0:
                    color = adjusted_color[100]
                else:
                    color = adjusted_color[100]

                if i == 0:
                    ax.scatter(xi[i], yi[i], color=color, s=75, edgecolors='k', linewidths=edge_width, label=str(u), alpha=0.7)
                    handles.append(mlines.Line2D([], [], color=adjusted_color[75], alpha=0.7, marker='o', linestyle='None', label=str(u)))
                else:
                    ax.scatter(xi[i], yi[i], color=color, s=75, edgecolors='k', linewidths=edge_width, alpha=0.7)

        ax.set_xlabel('PC1 (71,2 %)', fontsize='x-large')
        ax.set_ylabel('PC2 (10,4 %)', fontsize='x-large')
        ax.legend(loc='upper right', handles=handles, fontsize=12, markerscale=1.2)        
        plt.tight_layout()
        plt.show()

In [ ]:
# Fonction qui réalise l'ensemble de la PCA et des représentations
def fPCA(data, deriv, method, normalisation, age_var):
    nfeat, labs = extract_data(data, deriv, normalisation)
    Xt, pca = apply_PCA_4(nfeat, method)
    if age_var == True :
        plot_PCA_map_age_var(Xt, labs, data)
    elif age_var == False :
        plot_PCA_map_no_age_var(Xt, labs, data)
    test_PCA_nofig(data)
    return pca.components_

In [ ]:
# Représentation des charges fonctionnelles
def display_loadings(pca_components, features):
    n_components = pca_components.shape[0]
    n_components = 2
    Wavelength = data.columns[1:-4]
    Wavelength = Wavelength[::-1]
    plt.figure(figsize=(12/1.7, 3+n_components/1.7))
    colors1 = plt.cm.Set1(np.linspace(0, 0.5, n_components))

    for i in range(n_components):
        offset = i/10
        plt.plot(Wavelength, pca_components[i] - offset, color=colors1[i], linestyle='-', linewidth=0.5, alpha=1, label=f'PC{i+1}')
        plt.axhline(y=-offset, color='black', linestyle='--', linewidth=0.5)

    plt.xlabel('Wavelength')
    plt.ylabel('loading intensity')
    plt.gca().invert_xaxis()
    plt.legend()
    plt.show()

## Plot Figures

In [ ]:
# Figure 12 
data_SPECTRAA = data
fPCA(data, 1, 'SVD', 'normalise', False)

In [ ]:
# Figure 15 et Annexe 14
data_extract = data[data['compound'] == 'Caoutchouc']
display_loadings(fPCA(data_extract, 0, 'SVD', 'normalise', True), data_extract.columns[1:-4])

In [ ]:
# Annexe 15
data_extract = data[data['compound'] == 'Acrylique']
display_loadings(fPCA(data_extract, 0, 'SVD', 'normalise', True), data_extract.columns[1:-4])

In [ ]:
# Annexe 13
data_extract = data[data['compound'] == 'PP']
display_loadings(fPCA(data_extract, 0, 'SVD', 'normalise', True), data_extract.columns[1:-4])